<a href="https://colab.research.google.com/github/mafloan/TelecomChallengeAllura/blob/main/TelecomMorosos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nueva sección

In [ ]:
!pip install pandas matplotlib requests

In [ ]:
# Either run the command below or the one after it, but not both.
!python telecomx_pipeline.py --from raw
# Or local:
# !python telecomx_pipeline.py --from local --path ./TelecomX_Data.json

python3: can't open file '/content/telecomx_pipeline.py': [Errno 2] No such file or directory


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet

# Configuración de rutas
OUTPUT_DIR = "reports"
CSV_PATH = "outputs/telecomx_clean.csv"
MD_PATH = os.path.join(OUTPUT_DIR, "EDA_TelecomX.md")
PDF_PATH = os.path.join(OUTPUT_DIR, "EDA_TelecomX.pdf")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Cargar dataset
try:
    df = pd.read_csv(CSV_PATH)
except FileNotFoundError:
    raise FileNotFoundError("No se encontró el archivo limpio en outputs/. Corre primero telecomx_pipeline.py")

# 2. Resumen de datos
n_rows, n_cols = df.shape
summary = df.info()
nulos = df.isnull().sum().to_dict()

# 3. Distribución de churn
if "churn" in df.columns:
    churn_counts = df["churn"].value_counts()
    plt.figure()
    churn_counts.plot(kind="bar")
    plt.title("Distribución de Churn")
    plt.ylabel("Cantidad")
    churn_img = os.path.join(OUTPUT_DIR, "churn_dist.png")
    plt.savefig(churn_img)
    plt.close()
else:
    churn_counts = None
    churn_img = None

# 4. Variables categóricas top
cat_plots = []
for col in df.select_dtypes(include=["object"]).columns[:3]:
    plt.figure()
    df[col].value_counts().head(5).plot(kind="bar")
    plt.title(f"Top categorías en {col}")
    img_path = os.path.join(OUTPUT_DIR, f"cat_{col}.png")
    plt.savefig(img_path)
    plt.close()
    cat_plots.append(img_path)

# 5. Variables numéricas
num_plots = []
for col in df.select_dtypes(include=["int64", "float64"]).columns[:2]:
    plt.figure()
    df[col].hist(bins=30)
    plt.title(f"Distribución de {col}")
    img_path = os.path.join(OUTPUT_DIR, f"num_{col}.png")
    plt.savefig(img_path)
    plt.close()
    num_plots.append(img_path)

# 6. Guardar Markdown
with open(MD_PATH, "w", encoding="utf-8") as f:
    f.write("# Informe EDA - TelecomX\n\n")
    f.write(f"Filas: {n_rows}, Columnas: {n_cols}\n\n")
    f.write("## Valores nulos por columna\n")
    for k, v in nulos.items():
        f.write(f"- {k}: {v}\n")
    if churn_counts is not None:
        f.write("\n## Distribución de Churn\n")
        f.write(str(churn_counts.to_dict()))

# 7. Guardar PDF
styles = getSampleStyleSheet()
elements = []
doc = SimpleDocTemplate(PDF_PATH, pagesize=A4)
elements.append(Paragraph("<b>Informe EDA - TelecomX</b>", styles['Title']))
elements.append(Spacer(1, 12))
elements.append(Paragraph(f"Filas: {n_rows}, Columnas: {n_cols}", styles['Normal']))

# Nulos
elements.append(Paragraph("<b>Valores nulos por columna:</b>", styles['Heading2']))
for k, v in nulos.items():
    elements.append(Paragraph(f"{k}: {v}", styles['Normal']))

# Imágenes
def add_image(path, width=400):
    if path and os.path.exists(path):
        elements.append(Spacer(1, 12))
        elements.append(Image(path, width, width*0.6))

if churn_img:
    add_image(churn_img)
for img in cat_plots + num_plots:
    add_image(img)

doc.build(elements)

print(f"✅ Reportes generados: {MD_PATH} y {PDF_PATH}")


FileNotFoundError: No se encontró el archivo limpio en outputs/. Corre primero telecomx_pipeline.py

In [ ]:
!pip install reportlab

In [ ]:
🔹 1. Visualizar la matriz de correlación

Sirve para ver cómo se relacionan las variables numéricas entre sí y, sobre todo, con la variable objetivo (Churn o Cancelación).

import matplotlib.pyplot as plt
import seaborn as sns

# Correlación solo con variables numéricas
corr = df.corr(numeric_only=True)

plt.figure(figsize=(12,8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación", fontsize=16)
plt.show()

# Relación de cada variable con la variable objetivo
correlacion_churn = corr["Churn"].sort_values(ascending=False)
print("Correlación con la variable Churn:")
print(correlacion_churn)


👉 Aquí te interesa detectar variables con correlaciones más altas (positivas o negativas) con Churn.
Ejemplo:

tenure con correlación negativa (clientes antiguos cancelan menos).

MonthlyCharges positiva (cuotas altas → más cancelaciones).

🔹 2. Importancia de variables según modelo
📌 Regresión Logística

Los coeficientes (model.coef_) indican el peso de cada variable.

import pandas as pd
import numpy as np

coef = pd.DataFrame({
    "Variable": X.columns,
    "Coeficiente": model_log.coef_[0]
})
coef["Importancia_Abs"] = np.abs(coef["Coeficiente"])
coef = coef.sort_values(by="Importancia_Abs", ascending=False)

print(coef)

📌 KNN

KNN no da importancia directa de variables, pero podés:

Normalizar datos para que todas tengan el mismo peso.

Usar permutation importance (de sklearn.inspection):

from sklearn.inspection import permutation_importance

result = permutation_importance(model_knn, X_test, y_test, n_repeats=10, random_state=42)
importances = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": result.importances_mean
}).sort_values(by="Importancia", ascending=False)

print(importances)

📌 Random Forest

Muy usado para ranking de variables 🚀

importances = model_rf.feature_importances_
forest_importance = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": importances
}).sort_values(by="Importancia", ascending=False)

print(forest_importance)

# Visualización
sns.barplot(x="Importancia", y="Variable", data=forest_importance)
plt.title("Importancia de Variables - Random Forest")
plt.show()

📌 SVM

Para kernel lineal (SVC(kernel="linear")), podés ver coeficientes:

coef = pd.DataFrame({
    "Variable": X.columns,
    "Coeficiente": model_svm.coef_[0]
})
coef["Importancia_Abs"] = np.abs(coef["Coeficiente"])
coef = coef.sort_values(by="Importancia_Abs", ascending=False)

print(coef)


Si usás kernel no lineal (RBF, poly), mejor aplicar permutation importance igual que con KNN.

✅ Con esto, vas a tener:

Matriz de correlación inicial → exploración.

Ranking de variables por modelo → explicación y conclusiones.

¿Querés que te prepare un bloque de código integrado que corra todos estos análisis (correlación + importancia en cada modelo) y te devuelva tablas y gráficos listos para tu reporte?

You said:
ranking de variables por modelo
ChatGPT said:

¡Excelente! 🙌 Entonces vamos directo a lo que pedís: cómo obtener el ranking de variables por cada modelo.

Te paso un bloque de código que podés correr después de entrenar tus modelos (model_log, model_knn, model_rf, model_svm).

📌 Código para ranking de variables por modelo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance

# Diccionario donde iremos guardando los rankings
rankings = {}

# =============== 1. REGRESIÓN LOGÍSTICA ===============
coef = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": model_log.coef_[0]
})
coef["Importancia_Abs"] = np.abs(coef["Importancia"])
coef = coef.sort_values(by="Importancia_Abs", ascending=False)
rankings["Logística"] = coef[["Variable", "Importancia_Abs"]]


# =============== 2. KNN (Permutation Importance) ===============
result_knn = permutation_importance(model_knn, X_test, y_test, n_repeats=10, random_state=42)
imp_knn = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": result_knn.importances_mean
}).sort_values(by="Importancia", ascending=False)
rankings["KNN"] = imp_knn


# =============== 3. RANDOM FOREST ===============
imp_rf = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": model_rf.feature_importances_
}).sort_values(by="Importancia", ascending=False)
rankings["Random Forest"] = imp_rf


# =============== 4. SVM ===============
if hasattr(model_svm, "coef_"):  # si es kernel lineal
    imp_svm = pd.DataFrame({
        "Variable": X.columns,
        "Importancia": np.abs(model_svm.coef_[0])
    }).sort_values(by="Importancia", ascending=False)
else:  # si es kernel no lineal → usar permutation importance
    result_svm = permutation_importance(model_svm, X_test, y_test, n_repeats=10, random_state=42)
    imp_svm = pd.DataFrame({
        "Variable": X.columns,
        "Importancia": result_svm.importances_mean
    }).sort_values(by="Importancia", ascending=False)

rankings["SVM"] = imp_svm


# =============== VISUALIZACIÓN GENERAL ===============
for modelo, df_rank in rankings.items():
    plt.figure(figsize=(8,5))
    sns.barplot(x="Importancia", y="Variable", data=df_rank.head(10))
    plt.title(f"Top 10 Variables - {modelo}")
    plt.show()

📊 Qué vas a obtener

Una tabla con el ranking de variables para cada modelo.

Gráficos de barras con el Top 10 de las variables más relevantes.

👉 Pregunta clave:
¿Querés que lo armemos para que al final te dé un ranking consolidado (comparando qué variables aparecen más veces en el top de cada modelo), o preferís ver cada ranking modelo por modelo por separado?

You said:
sí
ChatGPT said:

¡Genial! 🙌 Entonces te armo la versión que genera:

Ranking por modelo (como antes).

Ranking consolidado → combinando los resultados de todos los modelos para ver qué variables son consistentes.

📌 Código con ranking consolidado
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance

# Diccionario donde iremos guardando los rankings
rankings = {}

# =============== 1. REGRESIÓN LOGÍSTICA ===============
coef = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": model_log.coef_[0]
})
coef["Importancia_Abs"] = np.abs(coef["Importancia"])
coef = coef.sort_values(by="Importancia_Abs", ascending=False)
rankings["Logística"] = coef[["Variable", "Importancia_Abs"]].rename(columns={"Importancia_Abs":"Importancia"})


# =============== 2. KNN (Permutation Importance) ===============
result_knn = permutation_importance(model_knn, X_test, y_test, n_repeats=10, random_state=42)
imp_knn = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": result_knn.importances_mean
}).sort_values(by="Importancia", ascending=False)
rankings["KNN"] = imp_knn


# =============== 3. RANDOM FOREST ===============
imp_rf = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": model_rf.feature_importances_
}).sort_values(by="Importancia", ascending=False)
rankings["Random Forest"] = imp_rf


# =============== 4. SVM ===============
if hasattr(model_svm, "coef_"):  # kernel lineal
    imp_svm = pd.DataFrame({
        "Variable": X.columns,
        "Importancia": np.abs(model_svm.coef_[0])
    }).sort_values(by="Importancia", ascending=False)
else:  # kernel no lineal → permutation importance
    result_svm = permutation_importance(model_svm, X_test, y_test, n_repeats=10, random_state=42)
    imp_svm = pd.DataFrame({
        "Variable": X.columns,
        "Importancia": result_svm.importances_mean
    }).sort_values(by="Importancia", ascending=False)

rankings["SVM"] = imp_svm


# =============== VISUALIZACIÓN INDIVIDUAL ===============
for modelo, df_rank in rankings.items():
    plt.figure(figsize=(8,5))
    sns.barplot(x="Importancia", y="Variable", data=df_rank.head(10))
    plt.title(f"Top 10 Variables - {modelo}")
    plt.show()


# =============== RANKING CONSOLIDADO ===============
# Normalizamos cada ranking (escala 0-1 para poder comparar)
consolidado = []
for modelo, df_rank in rankings.items():
    df_norm = df_rank.copy()
    df_norm["Importancia"] = df_norm["Importancia"] / df_norm["Importancia"].max()
    df_norm["Modelo"] = modelo
    consolidado.append(df_norm)

consolidado = pd.concat(consolidado)

# Promedio de importancia por variable
ranking_final = consolidado.groupby("Variable")["Importancia"].mean().sort_values(ascending=False).reset_index()

print("🔝 Ranking Consolidado de Variables:")
print(ranking_final.head(15))

# Visualización consolidada
plt.figure(figsize=(8,6))
sns.barplot(x="Importancia", y="Variable", data=ranking_final.head(10))
plt.title("Top 10 Variables - Ranking Consolidado")
plt.show()

📊 Qué vas a obtener:

Ranking y gráfico por cada modelo (Logística, KNN, RF, SVM).

Ranking consolidado → una tabla con el promedio normalizado de importancia de cada variable, mostrando cuáles son las más relevantes de manera consistente.

In [ ]:
Voy a suponer un dataset típico de churn de telecomunicaciones (como el de Telco Customer Churn de Kaggle). Obviamente, cuando vos corras el código con tus datos, las variables y el orden exacto pueden variar.

📊 Ejemplo de interpretación

Supongamos que el ranking consolidado devuelve este Top 10:

Variable	Importancia Promedio
Tenure	0.92
MonthlyCharges	0.88
Contract_TwoYear	0.75
InternetService	0.72
PaymentMethod	0.69
TechSupport	0.65
OnlineSecurity	0.63
Dependents	0.59
SeniorCitizen	0.54
StreamingTV	0.51
🔎 Interpretación

Tenure (Antigüedad del cliente)

Alta importancia en todos los modelos.

Correlación negativa con churn → los clientes más antiguos tienden a permanecer, mientras que los nuevos cancelan más.

MonthlyCharges (Cuota mensual)

Fuerte correlación positiva con churn.

Los clientes con cargos mensuales más altos son más propensos a cancelar, posiblemente por percepción de costo excesivo.

Contract (Tipo de contrato)

Contratos de dos años reducen la cancelación drásticamente (seguridad de permanencia).

Contratos mensuales tienen mayor riesgo de churn.

InternetService (Tipo de servicio de internet)

Clientes con fibra óptica presentan más cancelaciones en comparación con DSL o sin internet, posiblemente por problemas de costo/soporte.

PaymentMethod (Método de pago)

Pagos electrónicos automáticos suelen asociarse con menor churn.

Pagos manuales con tarjeta/cheque → mayor churn.

TechSupport y OnlineSecurity

Los clientes que cuentan con soporte técnico o seguridad online presentan menos cancelaciones.

Indican que servicios de valor agregado fidelizan.

Dependents (Dependientes en el hogar)

Hogares con dependientes cancelan menos (quizás por uso compartido del servicio).

SeniorCitizen (Adultos mayores)

Se observa un poco más de churn en clientes mayores, aunque no es tan fuerte como otras variables.

StreamingTV / StreamingMovies

Moderada importancia → clientes que usan streaming pueden estar más comprometidos, pero también sensibles a fallas o precio.

✅ Conclusión para el negocio

Factores protectores contra el churn: contratos largos, servicios adicionales (soporte técnico, seguridad online), método de pago automático, antigüedad del cliente.

Factores de riesgo: altas tarifas mensuales, contratos mensuales, clientes nuevos, uso de fibra óptica sin soporte adecuado.

Esto le da a la empresa palancas de acción:

Incentivar contratos más largos.

Ofrecer descuentos o beneficios a clientes con altas cuotas mensuales.

Mejorar soporte técnico y servicios extra.

Diseñar campañas específicas para clientes nuevos (primeros 6 meses son críticos).

In [ ]:
📌 Código completo con reporte automático
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance

# ====================================
# 1. Ranking por modelo
# ====================================
rankings = {}

# --- Regresión Logística ---
coef = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": model_log.coef_[0]
})
coef["Importancia_Abs"] = np.abs(coef["Importancia"])
coef = coef.sort_values(by="Importancia_Abs", ascending=False)
rankings["Logística"] = coef[["Variable", "Importancia_Abs"]].rename(columns={"Importancia_Abs":"Importancia"})

# --- KNN (Permutation Importance) ---
result_knn = permutation_importance(model_knn, X_test, y_test, n_repeats=10, random_state=42)
imp_knn = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": result_knn.importances_mean
}).sort_values(by="Importancia", ascending=False)
rankings["KNN"] = imp_knn

# --- Random Forest ---
imp_rf = pd.DataFrame({
    "Variable": X.columns,
    "Importancia": model_rf.feature_importances_
}).sort_values(by="Importancia", ascending=False)
rankings["Random Forest"] = imp_rf

# --- SVM ---
if hasattr(model_svm, "coef_"):  # kernel lineal
    imp_svm = pd.DataFrame({
        "Variable": X.columns,
        "Importancia": np.abs(model_svm.coef_[0])
    }).sort_values(by="Importancia", ascending=False)
else:  # kernel no lineal
    result_svm = permutation_importance(model_svm, X_test, y_test, n_repeats=10, random_state=42)
    imp_svm = pd.DataFrame({
        "Variable": X.columns,
        "Importancia": result_svm.importances_mean
    }).sort_values(by="Importancia", ascending=False)

rankings["SVM"] = imp_svm


# ====================================
# 2. Ranking consolidado
# ====================================
consolidado = []
for modelo, df_rank in rankings.items():
    df_norm = df_rank.copy()
    df_norm["Importancia"] = df_norm["Importancia"] / df_norm["Importancia"].max()
    df_norm["Modelo"] = modelo
    consolidado.append(df_norm)

consolidado = pd.concat(consolidado)

ranking_final = consolidado.groupby("Variable")["Importancia"].mean().sort_values(ascending=False).reset_index()

print("🔝 Ranking Consolidado de Variables:")
print(ranking_final.head(15))


# ====================================
# 3. Visualización consolidada
# ====================================
plt.figure(figsize=(8,6))
sns.barplot(x="Importancia", y="Variable", data=ranking_final.head(10))
plt.title("Top 10 Variables - Ranking Consolidado")
plt.show()


# ====================================
# 4. Mini-reporte automático
# ====================================
top_vars = ranking_final.head(10)["Variable"].tolist()

print("\n📋 Reporte Automático de Interpretación:\n")

for var in top_vars:
    if "tenure" in var.lower():
        print(f"- {var}: La antigüedad del cliente es clave. Clientes con más tiempo suelen cancelar menos.")
    elif "monthly" in var.lower() or "charge" in var.lower():
        print(f"- {var}: Los cargos mensuales altos incrementan la probabilidad de cancelación.")
    elif "contract" in var.lower():
        print(f"- {var}: El tipo de contrato influye: contratos largos reducen la cancelación, los mensuales aumentan el riesgo.")
    elif "payment" in var.lower():
        print(f"- {var}: El método de pago es relevante. Pagos automáticos suelen asociarse con menos churn.")
    elif "internet" in var.lower():
        print(f"- {var}: El tipo de servicio de internet impacta. Fibra óptica suele tener más cancelación que DSL.")
    elif "support" in var.lower() or "security" in var.lower():
        print(f"- {var}: Servicios adicionales como soporte técnico o seguridad online reducen el churn.")
    elif "senior" in var.lower():
        print(f"- {var}: Los clientes mayores tienden a cancelar un poco más que otros segmentos.")
    elif "depend" in var.lower():
        print(f"- {var}: Los clientes con dependientes en el hogar suelen permanecer más tiempo.")
    elif "stream" in var.lower():
        print(f"- {var}: El uso de streaming puede fidelizar, pero también aumenta la sensibilidad a precio y calidad.")
    else:
        print(f"- {var}: Variable relevante detectada, merece un análisis más detallado en el negocio.")

📊 Qué hace este código

Genera ranking de importancia por cada modelo.

Construye un ranking consolidado (promedio normalizado entre modelos).

Muestra un gráfico del Top 10 consolidado.

Produce un mini-reporte interpretado automáticamente, con explicaciones sobre las variables más importantes (tenure, charges, contract, etc.).